In [ ]:
import keras
from keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
from keras.applications import ResNet50

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
%matplotlib inline

### Load data

In [ ]:
img = load_img('food-binary/Dosa/img39.jpeg')
x = img_to_array(img)

In [ ]:
plt.imshow(x/255.)

In [ ]:
img_generator = ImageDataGenerator(rotation_range=90.,
                                   featurewise_center=True, 
                                   horizontal_flip=True,
                                   fill_mode='reflect',
                                   vertical_flip=True,
                                   zoom_range=0.4,
                                   featurewise_std_normalization=True,
                                   width_shift_range=20,
                                   height_shift_range=20,
                                   validation_split=0.2, rescale=1./255)

def get_batches(path, subset, gen=img_generator, 
                shuffle=True, batch_size=8, class_mode='categorical'): 
    return gen.flow_from_directory(path, target_size=(228,228), 
                                   class_mode=class_mode, shuffle=shuffle, batch_size=batch_size, subset=subset)

### Data Augmentation

In [ ]:
img = load_img('food-binary/Dosa/img101.jpeg')  
x = img_to_array(img)  
x = x.reshape((1,) + x.shape)  

In [ ]:
!pwd

In [ ]:
# create preview folder

In [ ]:
i = 0
for batch in img_generator.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

In [ ]:
# Check images in preview folder

### Transfer Learning with Image Augmentation

In [ ]:
from keras.applications import ResNet50

In [ ]:
train_generator = get_batches('food-binary/', 'training')
val_generator = get_batches('food-binary/', 'validation')

In [ ]:
base_model = ResNet50(include_top=False, input_shape=(228,228,3))

In [ ]:
batch_size=28

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)
m = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# m.summary()

In [ ]:
m.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
m.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=1,
        validation_data=val_generator,
        validation_steps=800 // batch_size)